# Homework 3: Relation extraction using distant supervision

In [1]:
__author__ = "Bill MacCartney"
__version__ = "CS224U, Stanford, Spring 2019"

## Contents

1. [Overview](#Overview)
1. [Set-up](#Set-up)
1. [Baseline](#Baseline)
1. [Homework questions](#Homework-questions)
  1. [Different model factory [1 point]](#Different-model-factory-[1-point])
  1. [Directional unigram features [2 points]](#Directional-unigram-features-[2-points])
  1. [The part-of-speech tags of the "middle" words [2 points]](#The-part-of-speech-tags-of-the-"middle"-words-[2-points])
  1. [Your original system [4 points]](#Your-original-system-[4-points])
1. [Bake-off [1 point]](#Bake-off-[1-point])

## Overview

This homework and associated bake-off are devoted to the developing really effective relation extraction systems using distant supervision. 

As with the previous assignments, this notebook first establishes a baseline system. The initial homework questions ask you to create additional baselines and suggest areas for innovation, and the final homework question asks you to develop an original system for you to enter into the bake-off.

## Set-up

See [the first notebook in this unit](rel_ext_01_task.ipynb#Set-up) for set-up instructions.

In [2]:
import os
import rel_ext
from sklearn.linear_model import LogisticRegression

As usual, we unite our corpus and KB into a dataset, and create some splits for experimentation:

In [3]:
rel_ext_data_home = os.path.join('data', 'rel_ext_data')

In [4]:
corpus = rel_ext.Corpus(os.path.join(rel_ext_data_home, 'corpus.tsv.gz'))

In [5]:
kb = rel_ext.KB(os.path.join(rel_ext_data_home, 'kb.tsv.gz'))

In [6]:
dataset = rel_ext.Dataset(corpus, kb)

You are not wedded to this set-up for splits. The bake-off will be conducted on a previously unseen test-set, so all of the data in `dataset` is fair game:

In [7]:
splits = dataset.build_splits(
    split_names=['tiny', 'train', 'dev'],
    split_fracs=[0.01, 0.79, 0.20],
    seed=1)

In [8]:
splits

{'tiny': Corpus with 3,474 examples; KB with 445 triples,
 'train': Corpus with 263,285 examples; KB with 36,191 triples,
 'dev': Corpus with 64,937 examples; KB with 9,248 triples,
 'all': Corpus with 331,696 examples; KB with 45,884 triples}

## Baseline

In [14]:
def simple_bag_of_words_featurizer(kbt, corpus, feature_counter):
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.middle.split(' '):
            feature_counter[word] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for word in ex.middle.split(' '):
            feature_counter[word] += 1
    return feature_counter

In [15]:
featurizers = [simple_bag_of_words_featurizer]

In [16]:
model_factory = lambda: LogisticRegression(fit_intercept=True, solver='liblinear')

In [17]:
baseline_results = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=featurizers,
    model_factory=model_factory,
    verbose=True)


relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.840      0.385      0.679        340       5716
author                    0.828      0.540      0.749        509       5885
capital                   0.655      0.200      0.450         95       5471
contains                  0.791      0.596      0.743       3904       9280
film_performance          0.785      0.573      0.731        766       6142
founders                  0.812      0.397      0.672        380       5756
genre                     0.714      0.147      0.403        170       5546
has_sibling               0.866      0.246      0.576        499       5875
has_spouse                0.872      0.322      0.650        594       5970
is_a                      0.687      0.229      0.491        497       5873
nationality               0.662      0.163      0.410        301       5677
parents     

Studying model weights might yield insights:

In [13]:
rel_ext.examine_model_weights(baseline_results)

Highest and lowest feature weights for relation adjoins:

     2.575 Córdoba
     2.455 Valais
     2.450 Taluks
     ..... .....
    -1.130 America
    -1.147 Gulf
    -1.189 India

Highest and lowest feature weights for relation author:

     3.301 author
     2.686 book
     2.458 books
     ..... .....
    -2.168 or
    -3.001 Daisy
    -3.137 1925

Highest and lowest feature weights for relation capital:

     3.655 capital
     1.846 km
     1.651 posted
     ..... .....
    -1.243 International
    -1.730 borough
    -2.043 includes

Highest and lowest feature weights for relation contains:

     2.827 third-largest
     2.363 bordered
     1.971 bounded
     ..... .....
    -2.368 rise
    -2.597 who
    -3.608 second-largest

Highest and lowest feature weights for relation film_performance:

     4.007 starring
     3.603 co-starring
     3.421 alongside
     ..... .....
    -2.233 Anjaani
    -2.233 Anjaana
    -2.350 Iruvar

Highest and lowest feature weights for relation fo

## Homework questions

Please embed your homework responses in this notebook, and do not delete any cells from the notebook. (You are free to add as many cells as you like as part of your responses.)

### Different model factory [1 point]

The code in `rel_ext` makes it very easy to experiment with other classifier models: one need only redefine the `model_factory` argument. This question asks you to assess a [Support Vector Classifier](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html).

__To submit:__ A call to `rel_ext.experiment` training on the 'train' part of `splits` and assessing on its `dev` part, with `featurizers` as defined above in this notebook and the `model_factory` set to one based in an `SVC` with `kernel='linear'` and all other arguments left with default values.

In [18]:
from sklearn.svm import SVC

In [15]:
model_factory = lambda: SVC(kernel='linear')
_ = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=featurizers,
    model_factory=model_factory,
    verbose=True)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.774      0.332      0.611        340       5716
author                    0.765      0.613      0.729        509       5885
capital                   0.667      0.295      0.532         95       5471
contains                  0.784      0.601      0.739       3904       9280
film_performance          0.751      0.623      0.721        766       6142
founders                  0.740      0.442      0.652        380       5756
genre                     0.608      0.282      0.494        170       5546
has_sibling               0.793      0.238      0.541        499       5875
has_spouse                0.848      0.357      0.665        594       5970
is_a                      0.594      0.260      0.473        497       5873
nationality               0.467      0.189      0.361        301       5677
parents     

### Directional unigram features [2 points]

The current bag-of-words representation makes no distinction between "forward" and "reverse" examples. But, intuitively, there is big difference between _X and his son Y_ and _Y and his son X_. This question asks you to modify `simple_bag_of_words_featurizer` to capture these differences. 

__To submit:__

1. A feature function `directional_bag_of_words_featurizer` that is just like `simple_bag_of_words_featurizer` except that it distinguishes "forward" and "reverse". To do this, you just need to mark each word feature for whether it is derived from a subject–object example or from an object–subject example. The precise nature of the mark you add for the two cases doesn't make a difference to the model.

2. The macro-average F-score on the `dev` set that you obtain from running `rel_ext.experiment` with `directional_bag_of_words_featurizer` as the only featurizer. (Aside from this, use all the default values for `experiment` as exemplified above in this notebook.)

3. `rel_ext.experiment` returns some of the core objects used in the experiment. How many feature names does the `vectorizer` have for the experiment run in the previous step? (Note: we're partly asking you to figure out how to get this value by using the sklearn documentation, so please don't ask how to do it on Piazza!)

In [19]:
# step 1: define directional_bag_of_words_featurizer
def directional_bag_of_words_featurizer(kbt, corpus, feature_counter):
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.middle.split(' '):
            feature_counter[(word, "forward")] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for word in ex.middle.split(' '):
            feature_counter[(word, "backward")] += 1
    return feature_counter

In [20]:
directional_featurizers = [directional_bag_of_words_featurizer]

In [21]:
model_factory = lambda: LogisticRegression(fit_intercept=True, solver='liblinear')

In [22]:
# step 2: get the macro-average F score using all default parameters+functions
p2_result = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=directional_featurizers,
    model_factory=model_factory,
    verbose=True
)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.848      0.426      0.708        340       5716
author                    0.848      0.593      0.781        509       5885
capital                   0.710      0.232      0.502         95       5471
contains                  0.822      0.652      0.781       3904       9280
film_performance          0.842      0.663      0.799        766       6142
founders                  0.794      0.405      0.666        380       5756
genre                     0.769      0.235      0.529        170       5546
has_sibling               0.863      0.253      0.582        499       5875
has_spouse                0.869      0.347      0.668        594       5970
is_a                      0.756      0.249      0.538        497       5873
nationality               0.670      0.223      0.478        301       5677
parents     

In [20]:
print("The macro-average F-score is", 0.609)

The macro-average F-score is 0.609


In [21]:
# step 3: # of feature names does the vectorizer have for the experiment run in the previous step
print("number of feature names from the previous steps", len(p2_result['vectorizer'].get_feature_names()))

number of feature names from the previous steps 40345


### The part-of-speech tags of the "middle" words [2 points]

Our corpus distribution contains part-of-speech (POS) tagged versions of the core text spans. Let's begin to explore whether there is information in these sequences, focusing on `middle_POS`.

__To submit:__

1. A feature function `middle_bigram_pos_tag_featurizer` that is just like `simple_bag_of_words_featurizer` except that it creates a feature for bigram POS sequences. For example, given 

  `The/DT dog/N napped/V`
  
   we obtain the list of bigram POS sequences
  
   `['<s> DT', 'DT N', 'N V', 'V </s>']`. 
   
   Don't forget the start and end tags, to model those environments properly!

2. The macro-average F-score on the `dev` set that you obtain from running `rel_ext.experiment` with `middle_bigram_pos_tag_featurizer` as the only featurizer. (Aside from this, use all the default values for `experiment` as exemplified above in this notebook.)

Note: To parse `middle_POS`, one splits on whitespace to get the `word/TAG` pairs. Each of these pairs `s` can be parsed with `s.rsplit('/', 1)`.

In [23]:
import nltk

In [24]:
# helper function
def create_bigram(ex):
    POS = []
    for word in ex.middle_POS.split(' '):
        if not word:
            continue
        term, tag = word.rsplit('/', 1)
        POS.append(tag)
    # add </s> and <s>
    POS.insert(0,'<s>')
    POS.append('</s>')
    # convert POS into a bigram:
    bigrams = list(nltk.bigrams(POS))    
    return bigrams

In [25]:
# step 1: define middle_bigram_pos_tag_featurizer
def middle_bigram_pos_tag_featurizer(kbt, corpus, feature_counter):
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        # parse the middle_POS and get the 
        bigrams = create_bigram(ex)
        bigram_list = []
        for i in bigrams:
            i = " ".join(i)
            #bigram_list.append(i)
            feature_counter[i] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        # parse the middle_POS and get the 
        bigrams = create_bigram(ex)
        bigram_list = []
        for i in bigrams:
            i = " ".join(i)
            #bigram_list.append(i)
            feature_counter[i] += 1
    return feature_counter

In [26]:
middle_featurizers = [middle_bigram_pos_tag_featurizer]

In [27]:
model_factory = lambda: LogisticRegression(fit_intercept=True, solver='liblinear')

In [28]:
p3_result = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers= middle_featurizers,
    model_factory=model_factory,
    verbose=True
)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.816      0.365      0.654        340       5716
author                    0.804      0.338      0.630        509       5885
capital                   0.486      0.179      0.362         95       5471
contains                  0.750      0.590      0.712       3904       9280
film_performance          0.726      0.453      0.648        766       6142
founders                  0.605      0.182      0.413        380       5756
genre                     0.667      0.188      0.442        170       5546
has_sibling               0.689      0.168      0.426        499       5875
has_spouse                0.768      0.268      0.559        594       5970
is_a                      0.555      0.173      0.385        497       5873
nationality               0.392      0.066      0.198        301       5677
parents     

In [28]:
print("macro-average F-score is:", 0.442)

macro-average F-score is: 0.442


### Your original system [4 points]

There are many options, and this could easily grow into a project. Here are a few ideas:

- Try out different classifier models, from `sklearn` and elsewhere.
- Add a feature that indicates the length of the middle.
- Augment the bag-of-words representation to include bigrams or trigrams (not just unigrams).
- Introduce features based on the entity mentions themselves. <!-- \[SPOILER: it helps a lot, maybe 4% in F-score. And combines nicely with the directional features.\] -->
- Experiment with features based on the context outside (rather than between) the two entity mentions — that is, the words before the first mention, or after the second.
- Try adding features which capture syntactic information, such as the dependency-path features used by Mintz et al. 2009. The [NLTK](https://www.nltk.org/) toolkit contains a variety of [parsing algorithms](http://www.nltk.org/api/nltk.parse.html) that may help.
- The bag-of-words representation does not permit generalization across word categories such as names of people, places, or companies. Can we do better using word embeddings such as [GloVe](https://nlp.stanford.edu/projects/glove/)?
- Consider adding features based on WordNet synsets. Here's a little code to get you started with that:
  ```
  from nltk.corpus import wordnet as wn
  dog_compatible_synsets = wn.synsets('dog', pos='n')
 ```

In [29]:
# import packages
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier

In [32]:
# directional_featurizers + decision tree with max_depth = 5
DecisionTreeClassifier_result = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=directional_featurizers,
    model_factory=lambda: DecisionTreeClassifier(max_depth=5),
    verbose=True)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.909      0.382      0.713        340       5716
author                    0.722      0.619      0.699        509       5885
capital                   0.846      0.232      0.553         95       5471
contains                  0.813      0.418      0.684       3904       9280
film_performance          0.877      0.409      0.713        766       6142
founders                  0.908      0.284      0.631        380       5756
genre                     0.686      0.141      0.387        170       5546
has_sibling               0.788      0.230      0.531        499       5875
has_spouse                0.815      0.327      0.627        594       5970
is_a                      0.811      0.155      0.439        497       5873
nationality               0.681      0.163      0.416        301       5677
parents     

In [30]:
# directional_featurizers + AdaBoostClassifier()
AdaBoost_result = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=directional_featurizers,
    model_factory=lambda: AdaBoostClassifier(),
    verbose=True)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.807      0.418      0.680        340       5716
author                    0.812      0.534      0.736        509       5885
capital                   0.553      0.274      0.459         95       5471
contains                  0.816      0.564      0.749       3904       9280
film_performance          0.822      0.591      0.763        766       6142
founders                  0.782      0.368      0.639        380       5756
genre                     0.652      0.176      0.424        170       5546
has_sibling               0.780      0.271      0.567        499       5875
has_spouse                0.770      0.365      0.630        594       5970
is_a                      0.707      0.233      0.503        497       5873
nationality               0.509      0.179      0.372        301       5677
parents     

## Bake-off [1 point]

For the bake-off, we will release a test set right after class on April 29. The announcement will go out on Piazza. You will evaluate your custom model from the previous question on these new datasets using the function `rel_ext.bake_off_experiment`. Rules:

1. Only one evaluation is permitted.
1. No additional system tuning is permitted once the bake-off has started.

To enter the bake-off, upload this notebook on Canvas:

https://canvas.stanford.edu/courses/99711/assignments/187248

The cells below this one constitute your bake-off entry.

People who enter will receive the additional homework point, and people whose systems achieve the top score will receive an additional 0.5 points. We will test the top-performing systems ourselves, and only systems for which we can reproduce the reported results will win the extra 0.5 points.

The bake-off will close at 4:30 pm on May 1. Late entries will be accepted, but they cannot earn the extra 0.5 points. Similarly, you cannot win the bake-off unless your homework is submitted on time.

In [33]:
# Enter your bake-off assessment code in this cell. 
# Please do not remove this comment.

AdaBoost_result = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=directional_featurizers,
    model_factory=lambda: AdaBoostClassifier(),
    verbose=False)

rel_ext_data_home_test = os.path.join(rel_ext_data_home, 'bakeoff3-rel_ext-test-data')
rel_ext.bake_off_experiment(AdaBoost_result, rel_ext_data_home_test)


relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.829      0.466      0.717        438       7122
author                    0.800      0.547      0.733        645       7329
capital                   0.642      0.374      0.561        115       6799
contains                  0.779      0.561      0.723       3808      10492
film_performance          0.832      0.593      0.770       1011       7695
founders                  0.801      0.399      0.666        444       7128
genre                     0.518      0.154      0.352        188       6872
has_sibling               0.786      0.272      0.571        717       7401
has_spouse                0.807      0.387      0.663        780       7464
is_a                      0.647      0.180      0.426        611       7295
nationality               0.500      0.172      0.362        383       7067
parents     

In [ ]:
# On an otherwise blank line in this cell, please enter
# your macro-average f-score (an F_0.5 score) as reported 
# by the code above. Please enter only a number between 
# 0 and 1 inclusive. Please do not remove this comment.
0.570